In [1]:
/pwd

u'/Users/gunnarkleemann/Dropbox/coursework/BerkeleyDataSciences_8_11/BerkeleyCODE/W209code/GenomeWeb/PrePrc'

In [2]:
## dictionaries for recoding 
# for clinical significance
ClinSigDict={0:0,1:255,2:1,3:2,4:-1,5:-2,6:6,7:7}

DisNmDict={}


##FUNCTIONS


# Melt data on column names list to serve as Id and value (melted) vars
# remove cols with zero values 
def Melton(splitCol, data):

    id_vars=data.columns.values.tolist()[0:splitCol]
    value_vars=data.columns.values.tolist()[splitCol:data.shape[1]]
    Melted=pd.melt(data, id_vars, value_vars)
    print 'lines before empty cell strip:' + str(len(Melted))
    Meltedfilt=Melted[Melted['value']==1]
    print 'lines After empty cell strip:' + str(len(Meltedfilt))
    Meltedfilt.to_csv('genomicDt.csv', sep='\t', encoding='utf-8', index=False)
    return Meltedfilt

#MAKE COLUMN NAMES UNIFORM # fix column names,
def fixDATA(df, useAlt):
    if '#CHROM' in df.columns:
        df.rename(columns={'#CHROM': 'CHROM'}, inplace=True)

    # remove the mutation address column if it is already there
    if 'Mut' in df.columns:
        df.drop(["Mut"], axis=1, inplace=True)

    #make all chrom titles uniform
    try:
        if len(df["CHROM"][1]) > 4:
            df["CHROM"] = df["CHROM"].apply(lambda x: x[3:len(x)])
    except:
        print "chr no len"

    # make mutation address column
    chr = df["CHROM"].map(str)
    #print df["POS"]
    pos = df["POS"].map(str)

    print useAlt

    if useAlt == 'yes':
        alt = df["ALT"].map(str)
        df['Mut'] = 'chr' + chr + '-' + pos + '-' + alt
        print 'added alt'
    else:
        df['Mut'] = 'chr' + chr + '-' + pos
        print 'added NO alt'
    return df


def FirstItem(str, sep):
    Item=str.split(sep)[0]
    return Item

#Extract and clean the Clincal severity (CLNSIG), and gene info (GENINFO) from the INFO columns
### Add CLNDBN
def GetClinDt(FiltList):
    for n in range(len(FiltList) - 1):
            Info = FiltList.INFO_y[n].split(';') # Parse Info data
            FiltList.loc[n, ('SnpDB')] = 'http://www.ncbi.nlm.nih.gov/snp/' + Info[0][3:len(Info[0])]  #gen the dbSNP link
            FiltList.loc[n, ('RSnum')] = Info[0][3:len(Info[0])]
            #THE INFO FIELDS ARE NOT IN CONSTANT PLACES
            #to find the right field, cycle through the elements of each list - the field changes position!
            # NOTE: the info fields can contain repeat fields
            # For the sake of this build I am taking the first
            #Info = FiltList.INFO_y[n].split(';') # Parse Info data
            for m in range(len(Info)):
                if Info[m][0:6] == 'CLNSIG':
                    ClinSig = Info[m][7:len(Info[m])] # take the first assertion
                    #print ClinSig
                    try:
                        ClinSig=FirstItem(ClinSig, '|')
                    except:
                        pass
                    try:
                        ClinSig=FirstItem(ClinSig, ',')
                    except:
                        pass
                    #NOTE from the clinvar file we only consider 2,3,4,5 the rest are removed (no risk data)
                    ##INFO=<ID=CLNSIG,Number=.,Type=String,Description="Variant Clinical Significance
                    # 0 - Uncertain significance, 1 - not provided, 2 - Benign, 3 - Likely benign, 4 - Like pathogenic, 5 - Pathogenic, 6 - drug response, 7 - histocompatibility, 255 - other">
                    
                    ## Recoding is handled in a separate function ##
                    #if ClinSig in ("255", "1", "6", "7"):
                    #if ClinSig == 255 or ClinSig == 1 or ClinSig == 6 or ClinSig == 7:
                    #    ClinSig=0
                        #print ClinSig
                    
                    FiltList.loc[n, ('Clinical_Significance')] = ClinSig

            for n in range(len(Info)):
                if Info[n][0:8] == 'GENEINFO':
                    str1=Info[n][9:len(Info[n])]
                    str2=FirstItem(str1, ':') #Remove number and take first item
                    FiltList.loc[n, ('gene')] = str2
            
            for n in range(len(FiltListMelt) - 1):
                Info = FiltListMelt.INFO_y[n].split(';') # Parse Info data
                for p in range(len(Info)):
                    if Info[p][0:6] == 'CLNDBN':
                        str1=Info[p][7:len(Info[p])]
                        str2=FirstItem(str1, ':') #Remove number and take first item
                        FiltList.loc[p, ('Disease')] = str2        
                        Disease=[]
                        #print Info
                        #print str2
                        try:
                            Disease=FirstItem(str2, '|')
                            #print Disease
                        except:
                            pass
                        try:
                            Disease=FirstItem(str2, ',')
                            #print Disease
                        except:
                            pass
                        #print Disease
                        #print ('/n')   
    return FiltList
        
# Transform the signficance values
def recode(SignifVal, RecodeDict={0:0,1:255,2:1,3:2,4:-1,5:-2,6:6,7:7}):
    # for a tricolor (and black) scale...
    #0 - Uncertain significance (keep 0 middle value)
    #1 - not provided, (recode 255 black)
    #2 - Benign, (recode 1)
    #3 - Likely benign, (recode 2)
    #4 - Likely pathogenic,  (recode -1)
    #5 - Pathogenic, (recode -2)
    #6 - drug response(?)
    #7 - histocompatibility(?)
    #RecodeDict={0:0,1:255,2:1,3:2,4:-1,5:-2,6:6,7:7} # pass this into function
    recodeVal=RecodeDict[SignifVal]
    return (recodeVal)
    

In [3]:
#IMPORT THE DATA
import numpy as np
import pandas as pd
## these files are also on S3
## https://s3-us-west-2.amazonaws.com/209genomics/20130606_sample_info2.csv
## https://s3-us-west-2.amazonaws.com/209genomics/ALL.chrMT.phase3_callmom.20130502.genotypes.vcf.txt
## https://s3-us-west-2.amazonaws.com/209genomics/clinvar_20150804.vcf.txt

Genomes='ALL.chrMT.phase3_callmom.20130502.genotypes.vcf.txt'
#MutRef='common_and_clinical.vcf.txt' # this one has no Mitochondrial X and Y data
SampleInfo='20130606_sample_info2.csv'
MutRef2='clinvar_20150804.vcf.txt' # this one HAS Mitochondrial X and Y data

SkipCt1=9
hdCt2=55
hdCt3=66


SampleInfo=pd.read_table(SampleInfo, sep=',')
Genomes=pd.read_table(Genomes, skiprows=SkipCt1)
#MutRef=pd.read_table(MutRef, skiprows=hdCt2, low_memory=False)
MutRef2=pd.read_table(MutRef2, skiprows=hdCt3, low_memory=False)
# the data from the genome files is wide format they have to be melted to make this work

IOError: File 20130606_sample_info2.csv does not exist

In [4]:
# melt the data and strip empty cells - 
GenomesMelt=Melton(9, Genomes)

AttributeError: 'str' object has no attribute 'columns'

In [52]:
#fix the names and add the MUT colum to use for joining
useAlt='yes'

MutFile_fx = fixDATA(MutRef2, useAlt) # use this file for MT data
Genomes_fx = fixDATA(Genomes, useAlt)
GenomesMelt_fx = fixDATA(GenomesMelt, useAlt)

yes
added alt
yes
added alt
chr no len
yes
added alt


In [53]:
#We need to put Genomes_fx into a file suitable for clustering 
# row ID is variant, column ID is person 

# one format is .PCL file (preclustering) suitable for Java treeview to cluster. 
# a simpler format is for clustering in R or using
Genomes_fx.drop(['CHROM','QUAL','ALT','REF','FILTER','INFO','FORMAT','ID'], axis=1, inplace=True)
Genomes_fx.columns

Index([u'POS', u'HG00096', u'HG00097', u'HG00099', u'HG00100', u'HG00101',
       u'HG00102', u'HG00103', u'HG00105', u'HG00106', 
       ...
       u'NA21129', u'NA21130', u'NA21133', u'NA21135', u'NA21137', u'NA21141',
       u'NA21142', u'NA21143', u'NA21144', u'Mut'],
      dtype='object', length=2536)

In [54]:
#clean up the HEATMAP MATRIX file

cols = Genomes_fx.columns.tolist()

Pos=Genomes_fx.iloc[:,0:1]

cols = cols[-1:] + cols[:-1]# move the mutation column into the first position

Genomes_fx=Genomes_fx[cols]

Genomes_fx.drop(['POS'], axis=1, inplace=True)

#Genomes_fx=Pos + Genomes_fx 

Genomes_fx.head(2)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Mut  HG00096  HG00097  HG00099  HG00100  HG00101  HG00102  HG00103  \
0  chrMT-10-C        0        0        0        0        0        0        0   
1  chrMT-16-T        0        0        0        0        0        0        0   

   HG00105  HG00106   ...     NA21128  NA21129  NA21130  NA21133  NA21135  \
0        0        0   ...           0        0        0        0        0   
1        0        0   ...           0        0        0        0        0   

   NA21137  NA21141  NA21142  NA21143  NA21144  
0        0        0        0        0        0  
1        0        0        0        0        0  

[2 rows x 2535 columns]

In [55]:
#Use the fixed clinvar file (MutFile) to get to the disease, clinical significance and gene information
#MutFile_fx.to_csv('ClinVar_fx.txt')
MutFile_fx.head(2)

CHROM      POS           ID REF ALT QUAL FILTER  \
0     1  1014143  rs786201005   C   T    .      .   
1     1  1014316  rs672601345   C  CG    .      .   

                                                INFO              Mut  
0  RS=786201005;RSPOS=1014143;dbSNPBuildID=144;SS...   chr1-1014143-T  
1  RS=672601345;RSPOS=1014319;dbSNPBuildID=142;SS...  chr1-1014316-CG

In [56]:
SampleInfo.head(1)

Sample Family ID Population           Population Description Gender  \
0  HG00096   HG00096        GBR  British in England and Scotland   male   

  Relationship Unexpected Parent/Child  Non Paternity Siblings Grandparents  \
0          NaN                      NaN           NaN      NaN          NaN   

  Avuncular Half Siblings Unknown Second Order Third Order  Other Comments  
0       NaN           NaN                  NaN         NaN             NaN

In [57]:
## Merge both INDIVIDUAL AND MATRIX data on the list of known mutations (ClivarList = MutFile_fx )
## This filter removes mutations that we have no information about and 
## puts the gene, clinical significance and gene data in register with the mutation (MUT) Column.

print 'lines before Clinvar filter:' + str(len(Genomes_fx))
#The heatmap data
FiltList = pd.merge(Genomes_fx, MutFile_fx, on=['Mut'], how='inner')
#The individual genome summary
FiltListMelt = pd.merge(GenomesMelt_fx, MutFile_fx, on=['Mut'], how='inner')
print 'lines After ClinVar filter:' + str(len(FiltList))
print str(len(np.unique(FiltList['Mut']))) + ' Unique Mutations'
print ('\n')
print 'heatmap (FiltList)'
print ('\n')
print FiltList.head(3)
print ('\n')
print 'Individual summary (FiltListMelt)'
print ('\n')
print FiltListMelt.head(3)

lines before Clinvar filter:3874
lines After ClinVar filter:111
111 Unique Mutations


heatmap (FiltList)


             Mut  HG00096  HG00097  HG00099  HG00100  HG00101  HG00102  \
0    chrMT-827-G        0        0        0        0        0        0   
1    chrMT-951-A        0        0        0        0        0        0   
2  chrMT-961-C,G        0        0        0        0        0        0   

   HG00103  HG00105  HG00106  \
0        0        0        0   
1        0        0        0   
2        0        0        0   

                         ...                          NA21143  NA21144  CHROM  \
0                        ...                                0        0     MT   
1                        ...                                0        0     MT   
2                        ...                                0        0     MT   

   POS           ID  REF  ALT  QUAL  FILTER  \
0  827   rs28358569    A    G     .       .   
1  951  rs200887992    G    A     .       .   


In [58]:
#Merge the sample info with INDIVUDUAL GENOME SUMMARY
#list on the genome(variable/Sample) feature and clean up

FiltListMelt.rename(columns={'variable':'Sample'}, inplace=True)
FiltListMelt = pd.merge(FiltListMelt, SampleInfo, on=['Sample'], how='inner')
FiltListMelt.drop(FiltListMelt.columns[24:len(FiltListMelt.columns)], axis=1, inplace= True)

#FiltList.drop(FiltList.columns[[2,17,18]], axis=1, inplace= True)
#FiltList.drop(FiltList.columns[[5,7,9]], axis=1, inplace= True)
FiltListMelt.drop(['ID_x','QUAL_x', 'FILTER_x', 'FORMAT','ID_y','QUAL_y', 'FILTER_y'], axis=1, inplace= True)

print FiltListMelt.columns
print FiltListMelt.shape

Index([u'CHROM_x', u'POS_x', u'REF_x', u'ALT_x', u'INFO_x', u'Sample',
       u'value', u'Mut', u'CHROM_y', u'POS_y', u'REF_y', u'ALT_y', u'INFO_y',
       u'Family ID', u'Population', u'Population Description', u'Gender'],
      dtype='object')
(10424, 17)


In [ ]:
##FOR THE INDIVIDUALS FILE - 
##ADD the clinical data to the individual file and save it.
#FiltListMelt.to_csv('1000genomesMtData')
FiltList2=GetClinDt(FiltListMelt)

#Output the data
out='INDIVgenomicDt'
datasrc='Mt'
Align='ALL.chrMT.phase3_callmom.20130502.genotypes.vcf.txt'
ref='common_and_clinical.vcf.txt'
fileNm = out + '_' + datasrc + '.csv'
FiltList2.to_csv(fileNm)

print FiltList2.columns
print FiltList2.shape

not_specified
/n
Familial_cancer_of_breast
/n
not_specified
/n
Familial_cancer_of_breast
/n
Leber's_optic_atrophy
/n
Leber's_optic_atrophy
/n
Familial_cancer_of_breast
/n
Neoplasm_of_ovary
/n
not_specified
/n
Familial_cancer_of_breast
/n
Familial_cancer_of_breast
/n
not_specified
/n
Familial_cancer_of_breast
/n
Brain_pseudoatrophy\x2c_reversible\x2c_valproate-induced\x2c_susceptibility_to
/n
Leber's_optic_atrophy
/n
Familial_cancer_of_breast
/n
not_specified
/n
Familial_cancer_of_breast
/n
Leber's_optic_atrophy
/n
Leber's_optic_atrophy
/n
Parkinson_disease\x2c_resistance_to
/n
not_specified
/n
Familial_cancer_of_breast
/n
not_specified
/n
Familial_cancer_of_breast
/n
Leber's_optic_atrophy
/n
Leber's_optic_atrophy
/n
Parkinson_disease\x2c_resistance_to
/n
not_specified
/n
Familial_cancer_of_breast
/n
Familial_cancer_of_breast
/n
not_specified
/n
Familial_cancer_of_breast
/n
Leber's_optic_atrophy
/n
Leber's_optic_atrophy
/n
Parkinson_disease\x2c_resistance_to
/n
not_specified
/n
Juvenile

In [60]:
##NOW FOCUS ON THE HEATMAP MATRIX## 
# Recode called mutations with thier clinical significance and 
# EITHER APPEND OR generate a second dataframe with disease and gene names. 


# The internal values are clincal significance and should be coded more intuitively
# ##INFO=<ID=CLNSIG,Number=.,Type=String,Description="Variant Clinical Significance, 

# for a tricolor scale...
#0 - Uncertain significance (keep 0 middle value)
#1 - not provided, (recode 255 black)
#2 - Benign, (recode 1)
#3 - Likely benign, (recode 2)
#4 - Likely pathogenic,  (recode -1)
#5 - Pathogenic, (recode -2)
#6 - drug response(?)
#7 - histocompatibility(?) 
# need to recode the non-muation sites 255 - Just Drop these 
## have 6, 7 and 255 been stripped by the fix data function?
FiltList.head(3)


Mut  HG00096  HG00097  HG00099  HG00100  HG00101  HG00102  \
0    chrMT-827-G        0        0        0        0        0        0   
1    chrMT-951-A        0        0        0        0        0        0   
2  chrMT-961-C,G        0        0        0        0        0        0   

   HG00103  HG00105  HG00106  \
0        0        0        0   
1        0        0        0   
2        0        0        0   

                         ...                          NA21143  NA21144  CHROM  \
0                        ...                                0        0     MT   
1                        ...                                0        0     MT   
2                        ...                                0        0     MT   

   POS           ID  REF  ALT  QUAL  FILTER  \
0  827   rs28358569    A    G     .       .   
1  951  rs200887992    G    A     .       .   
2  961    rs3888511    T  C,G     .       .   

                                                INFO  
0  RS=28358569;RSPOS=827;dbSNPBuildID=125;SSR=0;S...  
1  RS=200887992;RSPOS=951;dbSNPBuildID=136;SSR=0;...  
2  RS=3888511;RSPOS=961;dbSNPBuildID=108;SSR=0;SA...  

[3 rows x 2543 columns]

In [16]:
##split out the HEATMAP data, retain mutation 'mut' column
Mut_Matrix=FiltList.iloc[:,0:2535]

    #Mut_Matrix.drop('POS_x', axis=1, inplace=True)    
Mut_nparray = Mut_Matrix.as_matrix() # make a numpy array for searching
print Mut_nparray.shape
print Mut_nparray[1:5,]

(111, 2535)
[['chrMT-951-A' 0 0 ..., 0 0 0]
 ['chrMT-961-C,G' 0 0 ..., 0 0 0]
 ['chrMT-980-C' 0 0 ..., 0 0 0]
 ['chrMT-990-C' 0 0 ..., 0 0 0]]


In [17]:
# FIND THE INDEX OF THE MUTATIONS (non-zeros) and the WT sites (zeros)) 
# transpose is used to coerce the indexes into a usable shape
NonZeroIdx=np.transpose(np.nonzero(Mut_nparray[:,1:Mut_nparray.shape[1]]))
ZeroIdx=np.transpose(np.where(Mut_nparray == 0))
#adjust for the header row.
NonZeroIdx[:,1] = NonZeroIdx[:,1]+1
ZeroIdx[:,1] = ZeroIdx[:,1]+1
print NonZeroIdx[1:5,:]
print ZeroIdx[1:5,:]
print NonZeroIdx[:,1]

[[  0 224]
 [  0 271]
 [  0 347]
 [  0 353]]
[[0 3]
 [0 4]
 [0 5]
 [0 6]]
[ 202  224  271 ..., 2512 2519 2525]


NameError: name 'NonZeroIdx' is not defined

In [18]:
## RECODE the matrix replacing mutations with thier severity 
# replacing WT (0) with thier 255

# lookup the mutation - Get the severity score and Place it in the matrix 
for ii in range(len(NonZeroIdx)):
    MutID=Mut_nparray[NonZeroIdx[ii][0],0] # identify the mutation
    #print MutID + '_' + str(NonZeroIdx[z][0]) + "_" + str((NonZeroIdx[z][1])+1)
    row=np.where(FiltListMelt['Mut']==MutID)[0][0]
    ClinSig=int(FiltListMelt.iloc[row,19])
    #print ClinSig
    ClinSig=recode(ClinSig)
    #Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]=ClinSig
    Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]=ClinSig


In [19]:
NonZeroIdx[:,1]

array([ 202,  224,  271, ..., 2512, 2519, 2525])

In [ ]:
Mut_nparray.shape

(111, 2535)

In [ ]:
# Lookup WT alleles get position and code with 255
for j in range(len(ZeroIdx)):
    #MutID=Mut_nparray[ZeroIdx[j][0],0]
    #print MutID + '_' + str(NonZeroIdx[z][0]) + "_" + str((NonZeroIdx[z][1])+1)
    #row=np.where(FiltListMelt['Mut']==MutID)[0][0]  
    
    try:
        #Mut_Matrix.iloc[ZeroIdx[j][0],(ZeroIdx[j][1])]=255 #fill all of the WT positions
        print Mut_Matrix.iloc[ZeroIdx[j][0],(ZeroIdx[j][1])]
    except:
        # it is missing the last row...
        print ZeroIdx[j][0]
        print ZeroIdx[j][1]
        #print j

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-2
0
0
0
0
-2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-2
-2
-2
-2
0
0
0
0
0
0
0
0
0
0
0
0
-2
-2
0
0
0
0
0
0
0
0
-2
-2
0
-2
0
0
-2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-2
0
0
-2
0
0
0
-2
0
0
-2
0
0
0
-2
0
0
0
-2
0
0
0
0
0
0
0
0
0
0
0
-2
-2
0
0
0
-2
-2
0
0
0
0
-2
0
-2
0
-2
-2
-2
-2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [49]:
ZeroIdx[:,1]

array([   2,    3,    4, ..., 2533, 2534, 2535])

In [ ]:
## transpose matrix to put genotype on each row and add gender and populaton data for each sample
Mut_Matrix=Mut_Matrix.transpose()
Mut_Matrix.rename(columns = {'':'Sample'}, inplace = True)
Mut_Matrix.head()


In [ ]:
## add enthnicity column

#add
#FiltListMelt = pd.merge(FiltListMelt, SampleInfo['Populaton'], on=['Sample'], how='inner')


In [157]:
Mut_Matrix.rename(columns = {'':'Sample'}, inplace = True)


Mut HG00096 HG00097 HG00099 HG00100 HG00101 HG00102 HG00103  \
0    chrMT-827-G     255     255     255     255     255     255     255   
1    chrMT-951-A     255     255     255     255     255     255     255   
2  chrMT-961-C,G     255     255     255     255     255     255     255   
3    chrMT-980-C     255     255     255     255     255     255     255   
4    chrMT-990-C     255     255     255     255     255     255     255   

  HG00105 HG00106   ...   NA21128 NA21129 NA21130 NA21133 NA21135 NA21137  \
0     255     255   ...       255     255     255     255     255     255   
1     255     255   ...       255     255     255     255     255     255   
2     255     255   ...       255     255     255     255     255     255   
3     255     255   ...       255     255       2     255     255     255   
4     255     255   ...       255     255     255     255     255     255   

  NA21141 NA21142 NA21143 NA21144  
0     255     255     255     255  
1     255     255     255     255  
2     255     255     255     255  
3     255     255     255     255  
4     255     255     255     255  

[5 rows x 2535 columns]

In [ ]:
#Transpose to make individual genomes listed on rows rather than columns
Mut_Matrix=Mut_Matrix.transpose()
        #add ethnicity
#FiltListMelt = pd.merge(Mut_Matrix, SampleInfo['Population'], on=['Sample'], how='inner')
#Output the data
out='MatrixGenomDt'
datasrc='Mt'
fileNm = out + '_' + datasrc + '.csv'
Mut_Matrix.to_csv(fileNm, sep="\t", index_label="False", index="False")

In [119]:
###format data - http://blog.nextgenetics.net/?e=44###

In [153]:
FiltListMelt.head()

CHROM_x  POS_x REF_x ALT_x        INFO_x   Sample  value            Mut  \
0      MT   1438     A     G  VT=S;AC=2419  HG00096      1   chrMT-1438-G   
1      MT  15326     A     G  VT=S;AC=2521  HG00096      1  chrMT-15326-G   
2      MT   1438     A     G  VT=S;AC=2419  HG00097      1   chrMT-1438-G   
3      MT  15326     A     G  VT=S;AC=2521  HG00097      1  chrMT-15326-G   
4      MT   4216     T     C   VT=S;AC=139  HG00097      1   chrMT-4216-C   

  CHROM_y  POS_y  ...  ALT_y  \
0      MT   1438  ...      G   
1      MT  15326  ...      G   
2      MT   1438  ...      G   
3      MT  15326  ...      G   
4      MT   4216  ...      C   

                                              INFO_y Family ID Population  \
0  RS=2001030;RSPOS=1438;dbSNPBuildID=92;SSR=0;SA...   HG00096        GBR   
1  RS=2853508;RSPOS=15326;dbSNPBuildID=100;SSR=0;...   HG00096        GBR   
2  RS=2001030;RSPOS=1438;dbSNPBuildID=92;SSR=0;SA...   HG00097        GBR   
3  RS=2853508;RSPOS=15326;dbSNPBuildID=100;SSR=0;...   HG00097        GBR   
4  RS=1599988;RSPOS=4216;dbSNPBuildID=88;SSR=1;SA...   HG00097        GBR   

            Population Description  Gender  \
0  British in England and Scotland    male   
1  British in England and Scotland    male   
2  British in England and Scotland  female   
3  British in England and Scotland  female   
4  British in England and Scotland  female   

                                     SnpDB    RSnum Clinical_Significance  \
0  http://www.ncbi.nlm.nih.gov/snp/2001030  2001030                     2   
1  http://www.ncbi.nlm.nih.gov/snp/2853508  2853508                     4   
2  http://www.ncbi.nlm.nih.gov/snp/2001030  2001030                     2   
3  http://www.ncbi.nlm.nih.gov/snp/2853508  2853508                     4   
4  http://www.ncbi.nlm.nih.gov/snp/1599988  1599988                     0   

   gene  
0   NaN  
1  CYTB  
2   NaN  
3  CYTB  
4   ND1  

[5 rows x 21 columns]